In [1]:
import pandas as pd
empDf= pd.read_csv("employees.csv")
deptDf= pd.read_csv("departments.csv")
jobDf= pd.read_csv("jobs.csv")
jobHistDf= pd.read_csv("job_history.csv")
countryDf= pd.read_csv("countries.csv")
locationDf= pd.read_csv("locations.csv")

random questions:

#find the count of total employees per managers per department sorted by their count and department

In [ ]:
deptGrp= empDf.groupby(['DEPARTMENT_ID','MANAGER_ID'])
df1= deptGrp.size()
df2= df1.to_frame('size')
df3.sort_values(['size','DEPARTMENT_ID'], ascending= [False, True]).reset_index(drop=True)

#find the count of total employees per managers per department where salary>1000 and having count>2 sorted by their count and department 

In [ ]:
empDf[:2]
#applying where before group by
empPerDept= empDf[empDf['SALARY']>10000].groupby(['DEPARTMENT_ID','MANAGER_ID'])

#checking if group by has really happened by filtering on the dataframe before group by+normal filter vs after group by + filter(=having) 
empDf[empDf['SALARY']>10000]

#applying having part after group by
moreThan2Df= empPerDept.filter(lambda x: len(x)>=2)

moreThan2Df.groupby(['MANAGER_ID','DEPARTMENT_ID']).size().to_frame('size')

In [8]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [9]:
jobDf[:2]

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,AD_PRES,President,20080,40000
1,AD_VP,Administration Vice President,15000,30000


1. Display employees who joined in 2000 and doing job that has maximum salary more than 10000.

equivalent sql: 
select  first_name,job_id,salary, hire_date
from employees e join jobs 
on e.job_id=j.job_id
where to_char(e.hire_date,'yyyy') = 2000
and j.job_id j.min_salary>10000

In [99]:
empDf[:2]

In [100]:
jobDf[:2]

In [98]:
empJobDEtailDf= pd.merge(empDf, jobDf, how='inner', left_on= 'JOB_ID', right_on='JOB_ID')

In [27]:
empJobDEtailDf['HIRE_DATE']= pd.to_datetime(empJobDEtailDf['HIRE_DATE'])

In [28]:
empJobDEtailDf['mnth_yr']= empJobDEtailDf['HIRE_DATE'].apply(lambda x: x.strftime('%Y'))

In [51]:
empJobDEtailDf[(empJobDEtailDf['mnth_yr']=='2003') & (empJobDEtailDf['MIN_SALARY']>10000)]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY,_merge,mnth_yr
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,President,20080,40000,both,2003


In [116]:
empJobDEtailDf.dtypes

In [118]:
empJobDEtailDf.info()

3. Display jobs where the minimum salary is less than salary of employee 105

equivalent sql: 
select *
from  jobs
where min_salary<(select salary from employees
where employee_id=105);

pandas:

In [102]:
jobDf[:2]

In [103]:
empDf[:2]

In [62]:
emp105= empDf[empDf['EMPLOYEE_ID']==105]['SALARY']

In [65]:
emp105= emp105.to_frame('sal').reset_index(drop=True)

In [104]:
emp105

In [119]:
#ways of getting the value out of the df
# numpy.ndarray
emp105.values[0][0]
# pandas.core.series.Series
empDf.HIRE_DATE[0]

emp105.get_value(0,'sal')
emp105.ix[0,'sal']

In [182]:
#values vs df.col_name
# print(type(empDf.HIRE_DATE))
# print(type(empDf['HIRE_DATE'].values))
# print(empDf['HIRE_DATE'].values[0])
# print(empDf.HIRE_DATE[0])

<class 'pandas.core.series.Series'>


In [105]:
#can use anyone
jobDf[jobDf['MIN_SALARY']<emp105.ix[0,'sal']]
jobDf[jobDf['MIN_SALARY']<emp105.get_value(0,'sal')]
jobDf[jobDf['MIN_SALARY']<emp105.values[0][0]]

4. Display employees who have underscore in their email address

equivalent sql: 
select *
from employees 
where upper(email) like '%\_%';

In [ ]:
pandas:

In [120]:
empDf[:2]

In [113]:
empDf[empDf['EMAIL'].str.contains('_')]

5.Display employee name and manager name of the employee

equivalent sql:
select emp.first_name, emp.employee_id, mgr.first_name, mgr.employee_id
from employees emp left outer join employees mgr
on (emp.manager_id = mgr.employee_id)
order by emp.employee_id;

pandas:

In [112]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [121]:
empMgr= pd.merge(empDf, empDf, how='left', left_on='MANAGER_ID', right_on= 'EMPLOYEE_ID', suffixes=['_emp','_mgr'], indicator=True)

In [127]:
empMgr[['FIRST_NAME_emp','FIRST_NAME_mgr']]

6. Display number of employees joined in each year into department 30

sql:
select to_char(hire_date,'yyyy'),department_id, count(*)
from employees emp
where department_id=30
group by to_char(hire_date,'yyyy'),department_id

pandas:

In [128]:
empDf[:10]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,13-JAN-01,AD_VP,17000,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,03-JAN-06,IT_PROG,9000,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,21-MAY-07,IT_PROG,6000,NaN,103.0,60.0
5,105,David,Austin,DAUSTIN,590.423.4569,25-JUN-05,IT_PROG,4800,NaN,103.0,60.0
6,106,Valli,Pataballa,VPATABAL,590.423.4560,05-FEB-06,IT_PROG,4800,NaN,103.0,60.0
7,107,Diana,Lorentz,DLORENTZ,590.423.5567,07-FEB-07,IT_PROG,4200,NaN,103.0,60.0
8,108,Nancy,Greenberg,NGREENBE,515.124.4569,17-AUG-02,FI_MGR,12008,NaN,101.0,100.0
9,109,Daniel,Faviet,DFAVIET,515.124.4169,16-AUG-02,FI_ACCOUNT,9000,NaN,108.0,100.0


In [129]:
empDept30= empDf[empDf['DEPARTMENT_ID']==30]

In [132]:
empDept30['HIRE_YR']= pd.to_datetime(empDept30['HIRE_DATE']).apply(lambda x: x.strftime('%Y'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [136]:
perYrSize= empDept30.groupby('HIRE_YR').size().to_frame('count_of_emps')

In [138]:
perYrSize.reset_index()

,HIRE_YR,count_of_emps
0,2002,1
1,2003,1
2,2005,2
3,2006,1
4,2007,1


7. Display job id, department id and sum of salary by including al possible dimensions.

In [ ]:
pandas:

8. Display employee name and job title of jobs where salary of employee is between minimum and maximum salary for job.

sql:
select first_name, job_title
from employees e, jobs j
where salary between min_salary and max_salary
order by first_name;

pandas:

In [139]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [140]:
jobDf[:2]

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,AD_PRES,President,20080,40000
1,AD_VP,Administration Vice President,15000,30000


In [141]:
empJobDf= pd.merge(empDf, jobDf, how= 'inner', left_on= 'JOB_ID', right_on='JOB_ID', suffixes= ['_emp','_job'])

In [147]:
empJobDf[(empJobDf['SALARY']>empJobDf['MIN_SALARY']) | (empJobDf['SALARY']<empJobDf['MAX_SALARY'])]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY


9. Display how many employees have commission percentage and how many do not have

In [ ]:
sql:


In [ ]:
pandas:

In [157]:
empDf[:2]
#count of employees per commission pct 
empDf.groupby('COMMISSION_PCT').size()

# total count of employees
empDf['COMMISSION_PCT'].size
#employees with commission pct 
empDf['COMMISSION_PCT'].count()
#employees without commission pct
empDf['COMMISSION_PCT'].size- empDf['COMMISSION_PCT'].count()

72

10. Display first name, job title, department name of employees who joined on 28th Feb

sql:


pandas:

In [158]:
empDf[:10]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,13-JAN-01,AD_VP,17000,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,03-JAN-06,IT_PROG,9000,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,21-MAY-07,IT_PROG,6000,NaN,103.0,60.0
5,105,David,Austin,DAUSTIN,590.423.4569,25-JUN-05,IT_PROG,4800,NaN,103.0,60.0
6,106,Valli,Pataballa,VPATABAL,590.423.4560,05-FEB-06,IT_PROG,4800,NaN,103.0,60.0
7,107,Diana,Lorentz,DLORENTZ,590.423.5567,07-FEB-07,IT_PROG,4200,NaN,103.0,60.0
8,108,Nancy,Greenberg,NGREENBE,515.124.4569,17-AUG-02,FI_MGR,12008,NaN,101.0,100.0
9,109,Daniel,Faviet,DFAVIET,515.124.4169,16-AUG-02,FI_ACCOUNT,9000,NaN,108.0,100.0


In [183]:
empDf['yymm']= pd.to_datetime(empDf['HIRE_DATE']).apply(lambda x: x.strftime("%Y%m%d"))

In [185]:
# empDf
empDf[empDf['yymm']=='0228']

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yymm


Display details of jobs where the minimum salary is greater than 10000

sql:


pandas:

In [4]:
greaterThan1000= jobDf[jobDf['MIN_SALARY']>10000]

In [5]:
greaterThan1000

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,AD_PRES,President,20080,40000
1,AD_VP,Administration Vice President,15000,30000


12.Display the first name and join date of the employees who joined between 2002 and 2005

sql:


pandas:

In [8]:
empDf

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03,IT_PROG,9000,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,2007-05-21,IT_PROG,6000,NaN,103.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...
102,202,Pat,Fay,PFAY,603.123.6666,2005-08-17,MK_REP,6000,NaN,201.0,20.0
103,203,Susan,Mavris,SMAVRIS,515.123.7777,2002-06-07,HR_REP,6500,NaN,101.0,40.0
104,204,Hermann,Baer,HBAER,515.123.8888,2002-06-07,PR_REP,10000,NaN,101.0,70.0
105,205,Shelley,Higgins,SHIGGINS,515.123.8080,2002-06-07,AC_MGR,12008,NaN,101.0,110.0


In [21]:
empDf['HIRE_DATE']= pd.to_datetime(empDf['HIRE_DATE'])
empDf['only_yr']= empDf['HIRE_DATE'].apply(lambda x: x.strftime('%Y'))
empDf[empDf['only_yr'].between('2002','2005')][['FIRST_NAME','LAST_NAME']]

13.Display first name and join date of the employees who is either IT Programmer or Sales Man

sql:


pandas:


In [22]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005


In [23]:
jobDf[:2]

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,AD_PRES,President,20080,40000
1,AD_VP,Administration Vice President,15000,30000


In [24]:
empJobDf= pd.merge(empDf, jobDf, how='inner', left_on= 'JOB_ID', right_on= 'JOB_ID')

In [39]:
empJobDf[empJobDf['JOB_TITLE'].str.upper().str.contains('SALES') | empJobDf['JOB_TITLE'].str.upper().str.contains('IT')]

14. Display employees who joined after 1st January 2008

sql:


pandas:

In [40]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005


In [46]:
empDf['dd/mm/YYYY']= pd.to_datetime(pd.to_datetime(empDf['HIRE_DATE']).apply(lambda x: x.strftime('%d/%m/%Y')))

In [48]:
empDf[empDf['dd/mm/YYYY']>'2008-01-01']

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY
28,128,Steven,Markle,SMARKLE,650.124.1434,2008-03-08,ST_CLERK,2200,NaN,120.0,50.0,2008,2008-08-03
36,136,Hazel,Philtanker,HPHILTAN,650.127.1634,2008-02-06,ST_CLERK,2200,NaN,122.0,50.0,2008,2008-06-02
49,149,Eleni,Zlotkey,EZLOTKEY,011.44.1344.429018,2008-01-29,SA_MAN,10500,0.2,100.0,80.0,2008,2008-01-29
64,164,Mattea,Marvins,MMARVINS,011.44.1346.329268,2008-01-24,SA_REP,7200,0.1,147.0,80.0,2008,2008-01-24
65,165,David,Lee,DLEE,011.44.1346.529268,2008-02-23,SA_REP,6800,0.1,147.0,80.0,2008,2008-02-23
66,166,Sundar,Ande,SANDE,011.44.1346.629268,2008-03-24,SA_REP,6400,0.1,147.0,80.0,2008,2008-03-24
67,167,Amit,Banda,ABANDA,011.44.1346.729268,2008-04-21,SA_REP,6200,0.1,147.0,80.0,2008,2008-04-21
73,173,Sundita,Kumar,SKUMAR,011.44.1343.329268,2008-04-21,SA_REP,6100,0.1,148.0,80.0,2008,2008-04-21
79,179,Charles,Johnson,CJOHNSON,011.44.1644.429262,2008-01-04,SA_REP,6200,0.1,149.0,80.0,2008,2008-04-01
83,183,Girard,Geoni,GGEONI,650.507.9879,2008-02-03,SH_CLERK,2800,NaN,120.0,50.0,2008,2008-03-02


15. Display details of employee with ID 150 or 160

In [ ]:
sql:


pandas:

In [49]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-06-17
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-09-21


In [58]:
empDf[(empDf['EMPLOYEE_ID']==150) | (empDf['EMPLOYEE_ID']==160)]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY
50,150,Peter,Tucker,PTUCKER,011.44.1344.129268,2005-01-30,SA_REP,10000,0.3,145.0,80.0,2005,2005-01-30
60,160,Louise,Doran,LDORAN,011.44.1345.629268,2005-12-15,SA_REP,7500,0.3,146.0,80.0,2005,2005-12-15


16. Display first name, salary, commission pct, and hire date for employees with salary less than 1000

sql:


pandas:

In [61]:
empDf[empDf['SALARY']<10000]

17.Display job Title, the difference between minimum and maximum salaries for jobs with max salary in the range 10000 to 20000.

sql:
    

pandas:

In [65]:
jobDf[jobDf['MAX_SALARY'].between(10000,20000)][MIN_SALARY]

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
3,FI_MGR,Finance Manager,8200,16000
5,AC_MGR,Accounting Manager,8200,16000
8,SA_REP,Sales Representative,6000,12008
9,PU_MAN,Purchasing Manager,8000,15000
14,IT_PROG,Programmer,4000,10000
15,MK_MAN,Marketing Manager,9000,15000
18,PR_REP,Public Relations Representative,4500,10500


18.Display first name, salary, and round the salary to thousands.

pandas:

In [67]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-06-17
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-09-21


In [77]:
empDf['new_sal']=empDf['SALARY'].round(decimals=-2)
empDf[['FIRST_NAME','LAST_NAME']]

,FIRST_NAME,LAST_NAME
0,Steven,King
1,Neena,Kochhar
2,Lex,De Haan
3,Alexander,Hunold
4,Bruce,Ernst
...,...,...
102,Pat,Fay
103,Susan,Mavris
104,Hermann,Baer
105,Shelley,Higgins


19.Display details of jobs in the descending order of the title.

pandas:

In [80]:
jobDf.sort_values('JOB_TITLE', ascending=False)

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
11,ST_MAN,Stock Manager,5500,8500
12,ST_CLERK,Stock Clerk,2008,5000
13,SH_CLERK,Shipping Clerk,2500,5500
8,SA_REP,Sales Representative,6000,12008
7,SA_MAN,Sales Manager,10000,20080
9,PU_MAN,Purchasing Manager,8000,15000
10,PU_CLERK,Purchasing Clerk,2500,5500
18,PR_REP,Public Relations Representative,4500,10500
6,AC_ACCOUNT,Public Accountant,4200,9000
14,IT_PROG,Programmer,4000,10000


20.Display employees where the first name or last name starts with S.

pandas:

In [83]:
empDf[empDf['FIRST_NAME'].str.upper().str.contains('^S.*')]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY,new_sal
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-06-17,24000
16,116,Shelli,Baida,SBAIDA,515.127.4563,2005-12-24,PU_CLERK,2900,NaN,114.0,30.0,2005,2005-12-24,2900
17,117,Sigal,Tobias,STOBIAS,515.127.4564,2005-07-24,PU_CLERK,2800,NaN,114.0,30.0,2005,2005-07-24,2800
23,123,Shanta,Vollman,SVOLLMAN,650.123.4234,2005-10-10,ST_MAN,6500,NaN,100.0,50.0,2005,2005-10-10,6500
28,128,Steven,Markle,SMARKLE,650.124.1434,2008-03-08,ST_CLERK,2200,NaN,120.0,50.0,2008,2008-08-03,2200
38,138,Stephen,Stiles,SSTILES,650.121.2034,2005-10-26,ST_CLERK,3200,NaN,123.0,50.0,2005,2005-10-26,3200
61,161,Sarath,Sewall,SSEWALL,011.44.1345.529268,2006-11-03,SA_REP,7000,0.25,146.0,80.0,2006,2006-03-11,7000
66,166,Sundar,Ande,SANDE,011.44.1346.629268,2008-03-24,SA_REP,6400,0.10,147.0,80.0,2008,2008-03-24,6400
73,173,Sundita,Kumar,SKUMAR,011.44.1343.329268,2008-04-21,SA_REP,6100,0.10,148.0,80.0,2008,2008-04-21,6100
92,192,Sarah,Bell,SBELL,650.501.1876,2004-02-04,SH_CLERK,4000,NaN,123.0,50.0,2004,2004-04-02,4000


21.Display employees who joined in the month of May

pandas:

In [87]:
empDf

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY,new_sal
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-06-17,24000
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-09-21,17000
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000,NaN,100.0,90.0,2001,2001-01-13,17000
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03,IT_PROG,9000,NaN,102.0,60.0,2006,2006-03-01,9000
4,104,Bruce,Ernst,BERNST,590.423.4568,2007-05-21,IT_PROG,6000,NaN,103.0,60.0,2007,2007-05-21,6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,202,Pat,Fay,PFAY,603.123.6666,2005-08-17,MK_REP,6000,NaN,201.0,20.0,2005,2005-08-17,6000
103,203,Susan,Mavris,SMAVRIS,515.123.7777,2002-06-07,HR_REP,6500,NaN,101.0,40.0,2002,2002-07-06,6500
104,204,Hermann,Baer,HBAER,515.123.8888,2002-06-07,PR_REP,10000,NaN,101.0,70.0,2002,2002-07-06,10000
105,205,Shelley,Higgins,SHIGGINS,515.123.8080,2002-06-07,AC_MGR,12008,NaN,101.0,110.0,2002,2002-07-06,12000


In [104]:
empDf[empDf['HIRE_DATE'].astype(str).apply(lambda x: x[5:7])=='05']

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY,new_sal
4,104,Bruce,Ernst,BERNST,590.423.4568,2007-05-21,IT_PROG,6000,NaN,103.0,60.0,2007,2007-05-21,6000
15,115,Alexander,Khoo,AKHOO,515.127.4562,2003-05-18,PU_CLERK,3100,NaN,114.0,30.0,2003,2003-05-18,3100
22,122,Payam,Kaufling,PKAUFLIN,650.123.3234,2003-05-01,ST_MAN,7900,NaN,100.0,50.0,2003,2003-01-05,7900
74,174,Ellen,Abel,EABEL,011.44.1644.429267,2004-05-11,SA_REP,11000,0.30,149.0,80.0,2004,2004-11-05,11000
78,178,Kimberely,Grant,KGRANT,011.44.1644.429263,2007-05-24,SA_REP,7000,0.15,149.0,NaN,2007,2007-05-24,7000
97,197,Kevin,Feeney,KFEENEY,650.507.9822,2006-05-23,SH_CLERK,3000,NaN,124.0,50.0,2006,2006-05-23,3000


22.Display details of the employees where commission percentage is null and salary in the range 5000 to 10000 and department is 30.

pandas:

In [114]:
empDf[(empDf['COMMISSION_PCT'].isnull()) & (empDf['SALARY'].between(5000,10000))]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY,new_sal
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03,IT_PROG,9000,NaN,102.0,60.0,2006,2006-03-01,9000
4,104,Bruce,Ernst,BERNST,590.423.4568,2007-05-21,IT_PROG,6000,NaN,103.0,60.0,2007,2007-05-21,6000
9,109,Daniel,Faviet,DFAVIET,515.124.4169,2002-08-16,FI_ACCOUNT,9000,NaN,108.0,100.0,2002,2002-08-16,9000
10,110,John,Chen,JCHEN,515.124.4269,2005-09-28,FI_ACCOUNT,8200,NaN,108.0,100.0,2005,2005-09-28,8200
11,111,Ismael,Sciarra,ISCIARRA,515.124.4369,2005-09-30,FI_ACCOUNT,7700,NaN,108.0,100.0,2005,2005-09-30,7700
12,112,Jose Manuel,Urman,JMURMAN,515.124.4469,2006-03-07,FI_ACCOUNT,7800,NaN,108.0,100.0,2006,2006-07-03,7800
13,113,Luis,Popp,LPOPP,515.124.4567,2007-12-07,FI_ACCOUNT,6900,NaN,108.0,100.0,2007,2007-07-12,6900
20,120,Matthew,Weiss,MWEISS,650.123.1234,2004-07-18,ST_MAN,8000,NaN,100.0,50.0,2004,2004-07-18,8000
21,121,Adam,Fripp,AFRIPP,650.123.2234,2005-04-10,ST_MAN,8200,NaN,100.0,50.0,2005,2005-10-04,8200
22,122,Payam,Kaufling,PKAUFLIN,650.123.3234,2003-05-01,ST_MAN,7900,NaN,100.0,50.0,2003,2003-01-05,7900


23. Display first name and date of first salary of the employees.

pandas:

In [115]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY,new_sal
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-06-17,24000
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-09-21,17000


In [119]:
empDf['HIRE_DATE']= pd.to_datetime(empDf['HIRE_DATE'])

In [120]:
empDf['first_sal']= empDf['HIRE_DATE']+ pd.offsets.MonthBegin(1)

In [128]:
#checking of last month of the year gets converted to first day of next year
empDf[empDf['HIRE_DATE'].apply(lambda x: x.strftime('%m'))=='12']

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,only_yr,dd/mm/YYYY,new_sal,firstSalDay,first_sal
13,113,Luis,Popp,LPOPP,515.124.4567,2007-12-07,FI_ACCOUNT,6900,NaN,108.0,100.0,2007,2007-07-12,6900,2007-12-07,2008-01-01
14,114,Den,Raphaely,DRAPHEAL,515.127.4561,2002-12-07,PU_MAN,11000,NaN,100.0,30.0,2002,2002-07-12,11000,2002-12-07,2003-01-01
16,116,Shelli,Baida,SBAIDA,515.127.4563,2005-12-24,PU_CLERK,2900,NaN,114.0,30.0,2005,2005-12-24,2900,2005-12-24,2006-01-01
35,135,Ki,Gee,KGEE,650.127.1734,2007-12-12,ST_CLERK,2400,NaN,122.0,50.0,2007,2007-12-12,2400,2007-12-12,2008-01-01
54,154,Nanette,Cambrault,NCAMBRAU,011.44.1344.987668,2006-12-09,SA_REP,7500,0.2,145.0,80.0,2006,2006-09-12,7500,2006-12-09,2007-01-01
60,160,Louise,Doran,LDORAN,011.44.1345.629268,2005-12-15,SA_REP,7500,0.3,146.0,80.0,2005,2005-12-15,7500,2005-12-15,2006-01-01
91,191,Randall,Perkins,RPERKINS,650.505.4876,2007-12-19,SH_CLERK,2500,NaN,122.0,50.0,2007,2007-12-19,2500,2007-12-19,2008-01-01


24. Display first name and experience of the employees.

pandas:


In [9]:
empDf
empDf['HIRE_DATE']= pd.to_datetime(empDf['HIRE_DATE'])

In [10]:
#gives only days
empDf['exp']= pd.to_datetime('today')-empDf['HIRE_DATE']

0     6054 days 23:04:56.412739
1     5227 days 23:04:56.412739
2     6939 days 23:04:56.412739
3     5123 days 23:04:56.412739
4     4620 days 23:04:56.412739
                 ...           
102   5262 days 23:04:56.412739
103   6429 days 23:04:56.412739
104   6429 days 23:04:56.412739
105   6429 days 23:04:56.412739
106   6429 days 23:04:56.412739
Name: HIRE_DATE, Length: 107, dtype: timedelta64[ns]

In [47]:
from dateutil.relativedelta import relativedelta

In [105]:
#wont work as relativedelta works on single strings, hence lambda or apply or list actions on the two DF columns
diff= relativedelta.relativedelta(pd.to_datetime('today'),empDf['HIRE_DATE'])

strptime vs strftime:
https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

using of date formats:
https://stackabuse.com/converting-strings-to-datetime-in-python/

In [44]:
#example with relative delta approach
from datetime import datetime
from dateutil import relativedelta
date1 = datetime.strptime(str('2011-08-15 12:00:00'), '%Y-%m-%d %H:%M:%S')
date2 = datetime.strptime(str('2012-02-15'), '%Y-%m-%d')
r = relativedelta.relativedelta(date2, date1)

In [78]:
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd

In [98]:
def parse_date(date):
    return str(date.years)+' years '+str(date.years)+' months'

In [96]:
empDf['exp']=[parse_date(rd(datetime.strptime(str(i),'%Y-%m-%d %H:%M:%S.%f'),datetime.strptime(str(j),'%Y-%m-%d %H:%M:%S'))) for i,j in zip(empDf['date2'],empDf['HIRE_DATE'])]

25.Display first name of employees who joined in 2001.

In [107]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,date2,exp
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2020-01-14 00:27:55.994272,16 years 16
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2020-01-14 00:27:55.994272,14 years 14


In [109]:
empDf[empDf['HIRE_DATE'].astype(str).apply(lambda x: x[:4]=='2000')]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,date2,exp


26.Display first name and last name after converting the first letter of each name to upper case and the rest to lower case.

In [ ]:
pandas:


In [110]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,date2,exp
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2020-01-14 00:27:55.994272,16 years 16
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2020-01-14 00:27:55.994272,14 years 14


In [119]:
empDf['Full Name']=empDf['FIRST_NAME'].apply(lambda x: x[:1].upper()+x[1:].upper())+' '+empDf['LAST_NAME'].apply(lambda x: x[:1].upper()+x[1:].upper())

27. Display the first word in job title

In [ ]:
pandas:

In [120]:
jobDf[:2]

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY
0,AD_PRES,President,20080,40000
1,AD_VP,Administration Vice President,15000,30000


In [121]:
jobDf['First Word']= jobDf['JOB_TITLE'].apply(lambda x: x.split(' ')[0])

28. Display the length of first name for employees where last name contain character ‘b’ after 3rd position.

pandas:


In [124]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,date2,exp,Full Name
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2020-01-14 00:27:55.994272,16 years 16,STEVEN KING
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2020-01-14 00:27:55.994272,14 years 14,NEENA KOCHHAR


In [133]:
empDf[empDf['LAST_NAME'].apply(lambda x: 'b' in x[4:].lower())]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,date2,exp,Full Name
6,106,Valli,Pataballa,VPATABAL,590.423.4560,2006-02-05,IT_PROG,4800,NaN,103.0,60.0,2020-01-14 00:27:55.994272,13 years 13,VALLI PATABALLA
8,108,Nancy,Greenberg,NGREENBE,515.124.4569,2002-08-17,FI_MGR,12008,NaN,101.0,100.0,2020-01-14 00:27:55.994272,17 years 17,NANCY GREENBERG


29. Display first name in upper case and email address in lower case for employees where the first name and email address are same irrespective of the case

pandas:


In [4]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [17]:
empDf[empDf.apply(lambda row: str(row['FIRST_NAME'])==str(row['EMAIL']), axis=1)]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID


30.Display employees who joined in the current year

In [18]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [26]:
empDf['HIRE_DATE']= pd.to_datetime(empDf['HIRE_DATE'])
empDf[empDf['HIRE_DATE'].apply(lambda row: row.strftime('%Y')==pd.to_datetime('today').strftime('%Y'))]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID


31. Display the number of days between system date and 1st January 2011.

In [ ]:
pandas:


In [27]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0


In [33]:
a= pd.to_datetime('today')

In [29]:
from datetime import datetime

In [34]:
b= datetime.strptime('01-01-2011','%d-%m-%Y')

In [49]:
df1= pd.DataFrame([(a,b)],columns=['a','b'])
df1

,a,b
0,2020-01-14 12:04:35.623517,2011-01-01


In [51]:
from dateutil.relativedelta import relativedelta as rd
df1['diff']= df1.apply(lambda row: rd(row['a'],row['b']), axis=1)

32.Display how many employees joined in each month of the current year

pandas:


In [55]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0


In [60]:
empDf[empDf.apply(lambda row: row['HIRE_DATE'].strftime('%Y')==pd.to_datetime('today').strftime('%Y'), axis=1)]
# since no data in current year, hence taking 2003
df2003= empDf[empDf.apply(lambda row: row['HIRE_DATE'].strftime('%Y')=='2003', axis=1)]

In [63]:
df2003

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
15,115,Alexander,Khoo,AKHOO,515.127.4562,2003-05-18,PU_CLERK,3100,NaN,114.0,30.0
22,122,Payam,Kaufling,PKAUFLIN,650.123.3234,2003-05-01,ST_MAN,7900,NaN,100.0,50.0
37,137,Renske,Ladwig,RLADWIG,650.121.1234,2003-07-14,ST_CLERK,3600,NaN,123.0,50.0
41,141,Trenna,Rajs,TRAJS,650.121.8009,2003-10-17,ST_CLERK,3500,NaN,124.0,50.0
100,200,Jennifer,Whalen,JWHALEN,515.123.4444,2003-09-17,AD_ASST,4400,NaN,101.0,10.0


In [67]:
df2003['yyyymm']= df2003['HIRE_DATE'].apply(lambda x: x.strftime('%Y%m'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
countPerMnthDf= df2003.groupby('yyyymm').size().to_frame('count_per_mnth').reset_index(drop=False)

33.Display manager ID and number of employees managed by the manager sorted in descending order

In [93]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0


In [101]:
empDf.groupby('MANAGER_ID').size().to_frame('no_of_employees_managed').reset_index().sort_values(['no_of_employees_managed','MANAGER_ID'], ascending=[False,True])

34. Display employee ID and the date on which he ended his previous job

pandas:


In [116]:
jobHistDf.sort_values(['EMPLOYEE_ID','END_DATE'], ascending=[True,False])

,EMPLOYEE_ID,START_DATE,END_DATE,JOB_ID,DEPARTMENT_ID
1,101,21-SEP-97,27-OCT-01,AC_ACCOUNT,110
2,101,28-OCT-01,15-MAR-05,AC_MGR,110
0,102,13-JAN-01,24-JUL-06,IT_PROG,60
4,114,24-MAR-06,31-DEC-07,ST_CLERK,50
5,122,01-JAN-07,31-DEC-07,ST_CLERK,50
8,176,01-JAN-07,31-DEC-07,SA_MAN,80
7,176,24-MAR-06,31-DEC-06,SA_REP,80
9,200,01-JUL-02,31-DEC-06,AC_ACCOUNT,90
6,200,17-SEP-95,17-JUN-01,AD_ASST,90
3,201,17-FEB-04,19-DEC-07,MK_REP,20


In [113]:
jobHistDf.groupby('EMPLOYEE_ID')['END_DATE'].max().to_frame('last_job_date').reset_index()

,EMPLOYEE_ID,last_job_date
0,101,27-OCT-01
1,102,24-JUL-06
2,114,31-DEC-07
3,122,31-DEC-07
4,176,31-DEC-07
5,200,31-DEC-06
6,201,19-DEC-07


35.Display number of employees joined after 15th of the month.

pandas:


In [117]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


36.Display the country ID and number of cities we have in the country.

pandas:


In [8]:
locationDf[:2]

,LOCATION_ID,STREET_ADDRESS,POSTAL_CODE,CITY,STATE_PROVINCE,COUNTRY_ID
0,1000,1297 Via Cola di Rie,00989,Roma,NaN,IT
1,1100,93091 Calle della Testa,10934,Venice,NaN,IT


In [20]:
locationDf.groupby(['COUNTRY_ID'])['CITY'].count().to_frame('count_of_cities').reset_index().sort_values('count_of_cities', ascending=False).reset_index(drop=True)

37. Display average salary of employees in each department who have commission percentage.

pandas:


In [36]:
empDf[~empDf['COMMISSION_PCT'].isnull()].groupby('DEPARTMENT_ID')['SALARY'].mean()

DEPARTMENT_ID
80.0    8955.882353
Name: SALARY, dtype: float64

38. Display job ID, number of employees, sum of salary, and difference between highest salary and lowest salary of the employees of the job.

pandas:


In [40]:
agg_dict={'SALARY':['sum','max','min']}

In [46]:
salData= empDf.groupby('JOB_ID').agg(agg_dict).reset_index()

In [49]:
salData.info()
salData['diff']= salData[('SALARY','max')]-salData[('SALARY','min')]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
(JOB_ID, )       19 non-null object
(SALARY, sum)    19 non-null int64
(SALARY, max)    19 non-null int64
(SALARY, min)    19 non-null int64
dtypes: int64(3), object(1)
memory usage: 736.0+ bytes


39. Display job ID for jobs with average salary more than 10000.

pandas:

In [5]:
jobDf[:2]
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [18]:
meanSalPerJobDf= empDf.groupby('JOB_ID')['SALARY'].mean().to_frame('mean_value').reset_index()

In [22]:
meanSalPerJobDf[meanSalPerJobDf.apply(lambda x: x['mean_value']>10000, axis=1)]
# or, without using apply, i.e directly by DF operation
meanSalPerJobDf[meanSalPerJobDf['mean_value']>10000]

,JOB_ID,mean_value
1,AC_MGR,12008
3,AD_PRES,24000
4,AD_VP,17000
6,FI_MGR,12008
9,MK_MAN,13000
13,PU_MAN,11000
14,SA_MAN,12200


40. Display years in which more than 10 employees joined.

pandas:


In [29]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yr
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005


In [28]:
empDf['HIRE_DATE']= pd.to_datetime(empDf['HIRE_DATE'])
empDf['yr']= empDf.apply(lambda row: row['HIRE_DATE'].strftime('%Y'), axis=1)

In [31]:

joineesPerYrDf= empDf.groupby('yr').size().to_frame('count_of_emp_per_yr').reset_index()

In [39]:
filter doesnt work
# joineesPerYrDf.filter(lambda x: x['count_of_emp_per_yr']>2, axis=1)
joineesPerYrDf.apply(lambda x: x['count_of_emp_per_yr']>2, axis=1)
# or
joineesPerYrDf[joineesPerYrDf['count_of_emp_per_yr']>2]

,yr,count_of_emp_per_yr
1,2002,7
2,2003,6
3,2004,10
4,2005,29
5,2006,24
6,2007,19
7,2008,11


41.Display departments in which more than five employees have commission percentage.

pandas:

In [40]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yr
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005


In [43]:
empPerDept= empDf[~empDf['COMMISSION_PCT'].isnull()].groupby('DEPARTMENT_ID').size().to_frame('count_of_emp').reset_index()

In [44]:
empPerDept

,DEPARTMENT_ID,count_of_emp
0,80.0,34


42.Display employee ID for employees who did more than one job in the past

pandas:


In [45]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yr
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005


In [46]:
jobHistDf[:2]

,EMPLOYEE_ID,START_DATE,END_DATE,JOB_ID,DEPARTMENT_ID
0,102,13-JAN-01,24-JUL-06,IT_PROG,60
1,101,21-SEP-97,27-OCT-01,AC_ACCOUNT,110


In [47]:
empCountPerJob= jobHistDf.groupby('EMPLOYEE_ID').size().to_frame('count_of_emp').reset_index()

In [50]:
empCountPerJob[empCountPerJob['count_of_emp']>=2]

,EMPLOYEE_ID,count_of_emp
0,101,2
4,176,2
5,200,2


partially correct

In [54]:
df1= pd.merge(jobHistDf, empDf, how='inner', left_on= 'EMPLOYEE_ID', right_on='EMPLOYEE_ID', suffixes=['_jobHist','_emp'], indicator=True)

In [55]:
df1[df1['EMPLOYEE_ID']==101]

,EMPLOYEE_ID,START_DATE,END_DATE,JOB_ID_jobHist,DEPARTMENT_ID_jobHist,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID_emp,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID_emp,yr,_merge
1,101,21-SEP-97,27-OCT-01,AC_ACCOUNT,110,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,both
2,101,28-OCT-01,15-MAR-05,AC_MGR,110,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,both


43. Display job ID of jobs that were done by more than 3 employees for more than 100 days.

pandas:


In [70]:
#checking for a date+100
# empDf.apply(lambda x: print(x['HIRE_DATE'], x['HIRE_DATE']+pd.DateOffset(days=100)), axis=1) # faster
# # or
# from datetime import timedelta
# empDf.apply(lambda x: print(x['HIRE_DATE'], x['HIRE_DATE']+ timedelta(days=180)), axis=1)

2003-06-17 00:00:00 2003-09-25 00:00:00
2005-09-21 00:00:00 2005-12-30 00:00:00
2001-01-13 00:00:00 2001-04-23 00:00:00
2006-01-03 00:00:00 2006-04-13 00:00:00
2007-05-21 00:00:00 2007-08-29 00:00:00
2005-06-25 00:00:00 2005-10-03 00:00:00
2006-02-05 00:00:00 2006-05-16 00:00:00
2007-02-07 00:00:00 2007-05-18 00:00:00
2002-08-17 00:00:00 2002-11-25 00:00:00
2002-08-16 00:00:00 2002-11-24 00:00:00
2005-09-28 00:00:00 2006-01-06 00:00:00
2005-09-30 00:00:00 2006-01-08 00:00:00
2006-03-07 00:00:00 2006-06-15 00:00:00
2007-12-07 00:00:00 2008-03-16 00:00:00
2002-12-07 00:00:00 2003-03-17 00:00:00
2003-05-18 00:00:00 2003-08-26 00:00:00
2005-12-24 00:00:00 2006-04-03 00:00:00
2005-07-24 00:00:00 2005-11-01 00:00:00
2006-11-15 00:00:00 2007-02-23 00:00:00
2007-08-10 00:00:00 2007-11-18 00:00:00
2004-07-18 00:00:00 2004-10-26 00:00:00
2005-04-10 00:00:00 2005-07-19 00:00:00
2003-05-01 00:00:00 2003-08-09 00:00:00
2005-10-10 00:00:00 2006-01-18 00:00:00
2007-11-16 00:00:00 2008-02-24 00:00:00


0      None
1      None
2      None
3      None
4      None
       ... 
102    None
103    None
104    None
105    None
106    None
Length: 107, dtype: object

In [79]:
jobHistDf[:2]

,EMPLOYEE_ID,START_DATE,END_DATE,JOB_ID,DEPARTMENT_ID
0,102,13-JAN-01,24-JUL-06,IT_PROG,60
1,101,21-SEP-97,27-OCT-01,AC_ACCOUNT,110


In [81]:
jobHistDf['START_DATE']= pd.to_datetime(jobHistDf['START_DATE'])
jobHistDf['END_DATE']= pd.to_datetime(jobHistDf['END_DATE'])

In [88]:
#getting a new column as timedelta type with the difference of dates
jobHistDf['job_duration']= [(i-j) for i,j in zip(jobHistDf['END_DATE'],jobHistDf['START_DATE'])]

In [95]:
#finally getting results
jobHistDf[jobHistDf['job_duration']>pd.Timedelta(100,'D')]

,EMPLOYEE_ID,START_DATE,END_DATE,JOB_ID,DEPARTMENT_ID,job_duration
0,102,2001-01-13,2006-07-24,IT_PROG,60,2018 days
1,101,1997-09-21,2001-10-27,AC_ACCOUNT,110,1497 days
2,101,2001-10-28,2005-03-15,AC_MGR,110,1234 days
3,201,2004-02-17,2007-12-19,MK_REP,20,1401 days
4,114,2006-03-24,2007-12-31,ST_CLERK,50,647 days
5,122,2007-01-01,2007-12-31,ST_CLERK,50,364 days
6,200,1995-09-17,2001-06-17,AD_ASST,90,2100 days
7,176,2006-03-24,2006-12-31,SA_REP,80,282 days
8,176,2007-01-01,2007-12-31,SA_MAN,80,364 days
9,200,2002-07-01,2006-12-31,AC_ACCOUNT,90,1644 days


44.Display department ID, year, and Number of employees joined

pandas:


In [96]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yr,100DaysLater
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-09-25
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-12-30


In [98]:
empperYrPerDeptDf= empDf.groupby(['yr','DEPARTMENT_ID']).size().to_frame('count_of_emp').reset_index()
empperYrPerDeptDf[empperYrPerDeptDf['count_of_emp']>=2]

,yr,DEPARTMENT_ID,count_of_emp
4,2002,100.0,2
5,2002,110.0,2
8,2003,50.0,3
11,2004,50.0,4
12,2004,80.0,5
14,2005,30.0,2
15,2005,50.0,12
17,2005,80.0,10
19,2005,100.0,2
21,2006,50.0,13


45. Display departments where any manager is managing more than 5 employees

pandas:


In [101]:
empDf.iloc[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yr,100DaysLater
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-09-25
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-12-30


In [104]:
countOfEmpPerMgrDf= empDf.groupby('MANAGER_ID').size().to_frame('count_of_emp').reset_index()
countOfEmpPerMgrDf[countOfEmpPerMgrDf['count_of_emp']>=5]

46. Change salary of employee 115 to 8000 if the existing salary is less than 6000

pandas:


In [107]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,yr,100DaysLater
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0,2003,2003-09-25
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0,2005,2005-12-30


In [121]:
def changeSal(row):
    if row['EMPLOYEE_ID']==115:
        row['SALARY']= int(row['SALARY']) if int(row['SALARY'])>6000 else 8000
    return row

In [128]:
empDfNew= empDf.apply(changeSal, axis=1)
empDfNew[empDfNew['EMPLOYEE_ID']==115]

47. Insert a new employee into employees with all the required details.

pandas:


In [18]:
data=[[207, 'ANGELA', 'SNYDER','ANGELA','215 253 4737', 'a', 'SA_MAN', 12000, 80]]
cols= ['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'PHONE_NUMBER', 'HIRE_DATE','JOB_ID', 'SALARY', 'DEPARTMENT_ID']

In [19]:
df1= pd.DataFrame(data, columns=cols)
empDfNew= pd.concat([empDf,df1]).reset_index(drop=True)

In [25]:
empDfNew

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


48. Delete department 20

pandas:


In [27]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [40]:
empDf.drop(index= empDf[empDf['DEPARTMENT_ID']==20].index)
# or
empDf= empDf[empDf['DEPARTMENT_ID']!=20]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,13-JAN-01,AD_VP,17000,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,03-JAN-06,IT_PROG,9000,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,21-MAY-07,IT_PROG,6000,NaN,103.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...
100,200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,AD_ASST,4400,NaN,101.0,10.0
103,203,Susan,Mavris,SMAVRIS,515.123.7777,07-JUN-02,HR_REP,6500,NaN,101.0,40.0
104,204,Hermann,Baer,HBAER,515.123.8888,07-JUN-02,PR_REP,10000,NaN,101.0,70.0
105,205,Shelley,Higgins,SHIGGINS,515.123.8080,07-JUN-02,AC_MGR,12008,NaN,101.0,110.0


49. Change job ID of employee 110 to IT_PROG if the employee belongs to department 10 and the existing job ID does not start with IT.

pandas:


In [41]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [44]:
empDf[empDf['EMPLOYEE_ID']==110]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
10,110,John,Chen,JCHEN,515.124.4269,28-SEP-05,FI_ACCOUNT,8200,NaN,108.0,100.0


In [69]:
def change_job_id(row):
    if (row['DEPARTMENT_ID']==10)  & ('IT' not in row['JOB_ID'].upper()):
        return 'IT_PROG'
    else:
        return row['JOB_ID']

In [70]:
empDf['JOB_ID']=empDf.apply(change_job_id, axis=1)
empDf[empDf['DEPARTMENT_ID']==10]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
100,200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,IT_PROG,4400,NaN,101.0,10.0


In [72]:
empDf[empDf.apply(lambda row: 'SA' in row['JOB_ID'].upper(), axis=1)]

50. Insert a row into departments table with manager ID 120 and location ID in any location ID for city Tokyo

pandas:


In [46]:
deptDf[:2]
deptDf.info()
deptDf.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
DEPARTMENT_ID      27 non-null int64
DEPARTMENT_NAME    27 non-null object
MANAGER_ID         11 non-null float64
LOCATION_ID        27 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 992.0+ bytes


Index(['DEPARTMENT_ID', 'DEPARTMENT_NAME', 'MANAGER_ID', 'LOCATION_ID'], dtype='object')

In [45]:
locationDf[:2]
locationDf[locationDf['CITY'].str.upper()=='TOKYO']

,LOCATION_ID,STREET_ADDRESS,POSTAL_CODE,CITY,STATE_PROVINCE,COUNTRY_ID
2,1200,2017 Shinjuku-ku,1689,Tokyo,Tokyo Prefecture,JP


In [44]:
dataa=[[50,'test', 120.0, 1200]]
newDf= pd.DataFrame(data= dataa, columns= deptDf.columns)
pd.concat([newDf,deptDf]).reset_index(drop=True)

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,50,test,120.0,1200
1,10,Administration,200.0,1700
2,20,Marketing,201.0,1800
3,30,Purchasing,114.0,1700
4,40,Human Resources,203.0,2400
5,50,Shipping,121.0,1500
6,60,IT,103.0,1400
7,70,Public Relations,204.0,2700
8,80,Sales,145.0,2500
9,90,Executive,100.0,1700


In [43]:
newDf

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,50,test,120.0,1200


51. Display department name and number of employees in the department.

pandas:

In [55]:
deptDf[:2]
# empDf[:2]

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800


In [58]:
empPerDeptDf= empDf.groupby('DEPARTMENT_ID').size().to_frame('count_of_emp').reset_index()
deptDf.merge(empPerDeptDf, left_on= 'DEPARTMENT_ID', right_on= 'DEPARTMENT_ID', how='inner')[['DEPARTMENT_NAME','count_of_emp']]

,DEPARTMENT_NAME,count_of_emp
0,Administration,1
1,Marketing,2
2,Purchasing,6
3,Human Resources,1
4,Shipping,45
5,IT,5
6,Public Relations,1
7,Sales,34
8,Executive,3
9,Finance,6


52. Display job title, employee ID, number of days between ending date and starting date for all jobs in department 30 from job history.

pandas:
    

In [121]:
jobHistDf.info()
jobHistDf['START_DATE']= jobHistDf['START_DATE'].apply(lambda x: datetime.strptime(str(x),'%d-%b-%y'))
jobHistDf['END_DATE']= jobHistDf['END_DATE'].apply(lambda x: datetime.strptime(str(x),'%d-%b-%y'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
EMPLOYEE_ID      10 non-null int64
START_DATE       10 non-null object
END_DATE         10 non-null object
JOB_ID           10 non-null object
DEPARTMENT_ID    10 non-null int64
dtypes: int64(2), object(3)
memory usage: 528.0+ bytes


In [124]:
jobHistDf.info()
jobHistDf[:2]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
EMPLOYEE_ID      10 non-null int64
START_DATE       10 non-null datetime64[ns]
END_DATE         10 non-null datetime64[ns]
JOB_ID           10 non-null object
DEPARTMENT_ID    10 non-null int64
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 528.0+ bytes


,EMPLOYEE_ID,START_DATE,END_DATE,JOB_ID,DEPARTMENT_ID
0,102,2001-01-13,2006-07-24,IT_PROG,60
1,101,1997-09-21,2001-10-27,AC_ACCOUNT,110


In [129]:
from dateutil.relativedelta import relativedelta as rd

In [132]:
jobHistDf[['START_DATE','END_DATE']].apply(lambda x: print(x['START_DATE'], x['END_DATE'], rd(x['END_DATE'], x['START_DATE'])), axis=1)

2001-01-13 00:00:00 2006-07-24 00:00:00 relativedelta(years=+5, months=+6, days=+11)
1997-09-21 00:00:00 2001-10-27 00:00:00 relativedelta(years=+4, months=+1, days=+6)
2001-10-28 00:00:00 2005-03-15 00:00:00 relativedelta(years=+3, months=+4, days=+15)
2004-02-17 00:00:00 2007-12-19 00:00:00 relativedelta(years=+3, months=+10, days=+2)
2006-03-24 00:00:00 2007-12-31 00:00:00 relativedelta(years=+1, months=+9, days=+7)
2007-01-01 00:00:00 2007-12-31 00:00:00 relativedelta(months=+11, days=+30)
1995-09-17 00:00:00 2001-06-17 00:00:00 relativedelta(years=+5, months=+9)
2006-03-24 00:00:00 2006-12-31 00:00:00 relativedelta(months=+9, days=+7)
2007-01-01 00:00:00 2007-12-31 00:00:00 relativedelta(months=+11, days=+30)
2002-07-01 00:00:00 2006-12-31 00:00:00 relativedelta(years=+4, months=+5, days=+30)


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

In [131]:
jobHistDf[['START_DATE','END_DATE']]

,START_DATE,END_DATE
0,2001-01-13,2006-07-24
1,1997-09-21,2001-10-27
2,2001-10-28,2005-03-15
3,2004-02-17,2007-12-19
4,2006-03-24,2007-12-31
5,2007-01-01,2007-12-31
6,1995-09-17,2001-06-17
7,2006-03-24,2006-12-31
8,2007-01-01,2007-12-31
9,2002-07-01,2006-12-31


53. Display department name and manager first name

pandas:
join empDf(DEPARTMENT_ID) and deptDf witn

In [137]:
deptDf[:2]

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800


In [139]:
empDf[:2]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,21-SEP-05,AD_VP,17000,NaN,100.0,90.0


In [141]:
empDf.merge(deptDf, how='inner', left_on=['EMPLOYEE_ID','DEPARTMENT_ID'],right_on= ['MANAGER_ID','DEPARTMENT_ID'], suffixes=['_emp','_dept'])

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID_emp,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID_dept,LOCATION_ID
0,100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,NaN,NaN,90.0,Executive,100.0,1700
1,103,Alexander,Hunold,AHUNOLD,590.423.4567,03-JAN-06,IT_PROG,9000,NaN,102.0,60.0,IT,103.0,1400
2,108,Nancy,Greenberg,NGREENBE,515.124.4569,17-AUG-02,FI_MGR,12008,NaN,101.0,100.0,Finance,108.0,1700
3,114,Den,Raphaely,DRAPHEAL,515.127.4561,07-DEC-02,PU_MAN,11000,NaN,100.0,30.0,Purchasing,114.0,1700
4,121,Adam,Fripp,AFRIPP,650.123.2234,10-APR-05,ST_MAN,8200,NaN,100.0,50.0,Shipping,121.0,1500
5,145,John,Russell,JRUSSEL,011.44.1344.429268,01-OCT-04,SA_MAN,14000,0.4,100.0,80.0,Sales,145.0,2500
6,200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,AD_ASST,4400,NaN,101.0,10.0,Administration,200.0,1700
7,201,Michael,Hartstein,MHARTSTE,515.123.5555,17-FEB-04,MK_MAN,13000,NaN,100.0,20.0,Marketing,201.0,1800
8,203,Susan,Mavris,SMAVRIS,515.123.7777,07-JUN-02,HR_REP,6500,NaN,101.0,40.0,Human Resources,203.0,2400
9,204,Hermann,Baer,HBAER,515.123.8888,07-JUN-02,PR_REP,10000,NaN,101.0,70.0,Public Relations,204.0,2700
